In [14]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import swifter
from fastai.text import *
path = Path('/home/jupyter/tutorials/Thathasthu')
import gensim
from spellchecker import SpellChecker
spell = SpellChecker()

In [112]:
!pip install boto

In [24]:

train = pd.read_csv('./tathasthu/ttrain.csv')
manufacturers = pd.read_csv("./tathasthu/manufacturer_list.csv")


In [20]:
def clean_column(column, delimitter_list):
    column = column.str.lower()
    for delimitter in delimitter_list: 
        column = column.str.replace(delimitter," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    return column

def get_brand_name(article_name, choices):
    maxScore = 0
    maxChoice = ""
    for choice in choices:
        score = fuzz.token_set_ratio(" " + article_name + " ", " " + choice + " ")
        if score > maxScore or (score == maxScore and len(choice) > len(maxChoice)):
            maxScore = score
            maxChoice = choice
    return maxChoice, maxScore

def get_strict(article_name, choices):
    matches = []
    for choice in choices:
        choice1 = choice + " "
        choice2 = " " + choice + " "
        choice3 = " " + choice
        if article_name.startswith(choice1) or article_name.endswith(choice3) or choice2 in article_name:
            matches.append(choice)
    return matches

def remove_string(large, queries):
    for query in queries.split(" "):
        large = large.replace(query, "")
    large = large.replace("  ", " ")
    large = large.replace("  ", " ")
    large = large.replace("  ", " ")
    return large

def get_type(filtered_string):
    type_column = set()
    for word in filtered_string.split(' '):
        for seed in seed_choices:
            try: 
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add(word)
    #         print(seed + " " + word + " " + str(similarity_score))
    s = list(type_column)
    string_array = ' '.join(str(e) for e in s)
    return string_array

def get_words_from_sentence(sentence):
    words = sentence.split(" ")
    returns = []
    misspelled = spell.unknown(words)
    for word in words:
        if len(word) > 2 and word not in misspelled and word not in stopwords:
            returns.append(word)
    return returns

In [29]:
train['manufacturer_description'] = clean_column(train['manufacturer_description'], [])

In [31]:
train["company_name"] = train.manufacturer_description.str.extract(r"company name - ([a-zA-Z-09\. \&]+)")

In [34]:
train["article_name"] = clean_column(train['article_name'], ["'", "(", ")", "/"])

In [36]:
train['quantity'] = train['article_name'].str.extract(r'(\d.{0,4}(ml|g|kg|l|s|p))')[0]

In [38]:
manufacturers["brand_name"] = clean_column(manufacturers['brand_name'], ["'", "(", ")", "/"])

In [42]:
train["company_name"] = train["company_name"].fillna("")

In [46]:
train["article_name"] = train["article_name"].fillna("")

In [47]:
train['strict_transformed'] = train['article_name'].apply(lambda x: get_strict(x, manufacturers['brand_name']))

In [ ]:
train['strict_company'] = train['company_name'].apply(lambda x: get_strict(x, manufacturers['brand_name']))

In [49]:
train = train[train["article_name"] != ""]

In [50]:
train

,article_name,manufacturer_description,Unnamed: 2,Unnamed: 3,Unnamed: 4,company_name,quantity,strict_transformed,strict_company
0,chicken burger patty 250g,company name - venkys india ltd<br />membershi...,NaN,NaN,NaN,venkys india ltd,250g,[],[venkys]
1,britannia daily family long bread,company name - britannia industries ltd<br />m...,NaN,NaN,NaN,britannia industries ltd,NaN,[britannia],[britannia]
2,pooof potato kettle chips zesty masala pp 30g,company name - future consumer enterprise limi...,NaN,NaN,NaN,future consumer enterprise limited,30g,[],[]
3,fevibond tube 20 ml,membership discount - applicable<br />identifi...,NaN,NaN,NaN,,20 ml,[fevibond],[]
4,kitkat dessert delight truffle 50g,company name - nestle india ltd<br />membershi...,NaN,NaN,NaN,nestle india ltd,50g,[],[nestle]
5,blue bird spaghetti pasta 200g,company name - blue bird foods india pvt ltd<b...,NaN,NaN,NaN,blue bird foods india pvt ltd,200g,[blue bird],[blue bird]
6,nirapara avalose podi 500gm,company name - k k r food products<br />member...,NaN,NaN,NaN,k k r food products,500g,[nirapara],[]
7,nivea roll on fresh natural 50ml,membership discount - applicable<br />identifi...,NaN,NaN,NaN,,50ml,[nivea],[]
8,fiama dw soap exotic dream bx 115g,company name - itc india ltd<br />membership d...,NaN,NaN,NaN,itc india ltd,115g,[],[]
9,sangis ktcn peri peri mayonnaise bt 200g,company name - future consumer ltd<br />member...,NaN,NaN,NaN,future consumer ltd,200g,[],[]


In [51]:
train['fuzzy_transformed'] = train['article_name'].swifter.apply(lambda x: get_brand_name(x, manufacturers['brand_name']))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
train['fuzzy_company'] = train['company_name'].swifter.apply(lambda x: get_brand_name(x, manufacturers['brand_name']))

In [54]:
train = train.filter(['article_name','company_name', 'quantity', 'strict_transformed', 'strict_company', 'fuzzy_transformed', 'fuzzy_company']).join(train.fuzzy_transformed.apply(lambda loc: pd.Series(loc, index=['fuzzy_brand', 'brand_score']))).join(train.fuzzy_company.apply(lambda loc: pd.Series(loc, index=['f_company', 'company_score'])))



In [55]:
del train['fuzzy_transformed']
del train['fuzzy_company']

In [56]:
train["final_brand"] = train.swifter.apply(lambda x: x.fuzzy_brand if x.brand_score > 50 else x.strict_company[0] if len(x.strict_company) > 0 else None, axis=1)


In [57]:
train = train.filter(["article_name", "quantity", "final_brand"])

In [65]:
train["filtered"] = train.apply(lambda x: x.article_name if x.final_brand is None else remove_string(x.article_name, x.final_brand), axis=1)


In [68]:
train["filtered"] = train.apply(lambda x: remove_string(x.filtered, str(x.quantity)), axis=1)

In [71]:
train.to_csv("train_quantity.csv", index=False)

In [73]:
learn = load_learner(path)

In [86]:
train["final_brand"] = train.swifter.apply(lambda x: "" if x.final_brand == None else x.final_brand, axis=1)

Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 46619.42it/s]


In [88]:
train['function_dl'] = train.swifter.apply(lambda x: str(learn.predict(x['final_brand']+' '+x['filtered'])[0]), axis=1)

Pandas Apply: 100%|██████████| 8621/8621 [06:53<00:00, 20.83it/s]


In [89]:
function_df = pd.read_csv("./classes")

In [91]:
function_filtered = pd.merge(train,function_df, how='left', left_on = 'function_dl', right_on = 'class1')

In [92]:
function_filtered['class1'] = clean_column(function_filtered['class1'], ["'", "(", ")", "/", "&", ","])

In [93]:
function_filtered['class2'] = clean_column(function_filtered['class2'], ["'", "(", ")", "/", "&", ","])

In [94]:
function_filtered['appended_class'] = function_filtered.apply(lambda x: x.class1 + " " + x.class2, axis=1)

In [95]:
function_filtered["cleaned"] = function_filtered.apply(lambda x: remove_string(x.filtered, x.appended_class), axis=1)

In [96]:
final1_data = function_filtered.filter(['cleaned', 'quantity', 'final_brand', 'class1', 'class2'])

In [98]:
final1_data.to_csv('./final1_data.csv', index=False)

In [2]:
final1_data = pd.read_csv('./final1_data.csv')

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('./google.bin', binary=True)  



In [5]:
seed_choices = ['oranges', 'apples', 'avocado', 'onion', 'tomato', 'pants', 'shirt', 'pink', 'blue']
get_type("kitkat dessert delight truffle")

'truffle dessert'

In [9]:
final1_data["cleaned"] = final1_data.cleaned.fillna("")

In [10]:
final1_data['type'] = final1_data.swifter.apply(lambda x:get_type(x.cleaned), axis=1)


Pandas Apply: 100%|██████████| 8621/8621 [00:08<00:00, 1047.41it/s]

In [11]:
final1_data

,cleaned,quantity,final_brand,class1,class2,type
0,chicken burger patty,250g,venkys,ready to eat,branded food,patty chicken burger
1,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,bread
2,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,zesty potato
3,tube,20 ml,fevibond,otc,beauty personal care,
4,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,truffle dessert
5,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,pasta
6,avalose podi m,500g,nirapara,toys,toys stationery,
7,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,
8,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,
9,sangis ktcn peri peri mayonnaise bt,200g,NaN,jams spreads honey,branded food,mayonnaise


In [12]:
final1_data['cleaned_more'] = final1_data.swifter.apply(lambda x: remove_string(x.cleaned, x.type), axis=1)


Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 39572.37it/s]

In [13]:
final1_data

,cleaned,quantity,final_brand,class1,class2,type,cleaned_more
0,chicken burger patty,250g,venkys,ready to eat,branded food,patty chicken burger,
1,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,bread,daily family long
2,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,zesty potato,pooof kettle masala pp
3,tube,20 ml,fevibond,otc,beauty personal care,,tube
4,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,truffle dessert,kitkat delight
5,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,pasta,spaghetti
6,avalose podi m,500g,nirapara,toys,toys stationery,,avalose podi m
7,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,,roll on fresh natural
8,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,,dw soap exotic dream bx
9,sangis ktcn peri peri mayonnaise bt,200g,NaN,jams spreads honey,branded food,mayonnaise,sangis ktcn peri peri bt


In [16]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [21]:
final1_data["get_cleaned"] = final1_data.swifter.apply(lambda x: get_words_from_sentence(x.cleaned_more), axis=1)


Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 36105.80it/s]

In [22]:
final1_data

,cleaned,quantity,final_brand,class1,class2,type,cleaned_more,get_cleaned
0,chicken burger patty,250g,venkys,ready to eat,branded food,patty chicken burger,,[]
1,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,bread,daily family long,"[daily, family, long]"
2,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,zesty potato,pooof kettle masala pp,"[kettle, masala]"
3,tube,20 ml,fevibond,otc,beauty personal care,,tube,[tube]
4,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,truffle dessert,kitkat delight,[delight]
5,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,pasta,spaghetti,[spaghetti]
6,avalose podi m,500g,nirapara,toys,toys stationery,,avalose podi m,[]
7,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,,roll on fresh natural,"[roll, fresh, natural]"
8,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,,dw soap exotic dream bx,"[soap, exotic, dream]"
9,sangis ktcn peri peri mayonnaise bt,200g,NaN,jams spreads honey,branded food,mayonnaise,sangis ktcn peri peri bt,"[peri, peri]"


In [23]:
final1_data.to_csv("./final_data.csv", index=False)